#  Introductions to constrainat-based modeling using cobrapy

## Part 2: Flux Balance Analysis

### Instructor:
* Miguel Ponce de León from (Barcelona Supercomputing Center)
* Contact: miguel.ponce@bsc.es

11 December, 2020

# Part 2: Genome-scale modelling.

In this part we are gonna use a genome-scale metabolic model of Escherichia coli named iJO1366
The file has already been stored in the data folder and its path is data/iJO1366.xml

Alternatively, you can also access it here:
- [http://bigg.ucsd.edu/models/iJO1366](http://bigg.ucsd.edu/models/iJO1366)

to download the model and to see other metadata (citation, description, etc)

## Part 2.1: Studying the model.

### Read the SBML model

First we need to import the function read_sbml_model from the cobra.io modules

In [32]:
import cobra
from cobra.io import read_sbml_model

# State the path to the file iJO1366.xml
sbml_fname = './data/iJO1366.xml'

# Read the model
model = read_sbml_model(sbml_fname)

## Exercise 2.1: Inspecting the model's numbers

How many metabolites genes and reactions are contained in the model?

In [33]:
## TODO
## Write your code below
print('Total reactions: ', len(model.reactions))
print('Total number of metabolites: ', len(model.metabolites))
print('Total number of genes: ', len(model.genes))


Total reactions:  2583
Total number of metabolites:  1805
Total number of genes:  1367


In [34]:
# Inspecting the genes
# 1. Access a particular reaction:
#    * You can do it directly with: gene = model.genes.b0720
#    * Or you can use the function get_by_id: gene = model.genes.get_by_id('b0720')

b0720 = model.genes.b0720

b0722 = model.genes.get_by_id('b0722')

# 6. Inspect the reaction by printing:
#    1. gene.name
#    2. gene.id
#    3. gene.reactions

print('gene name: ', b0720.name)
print('gene id: ', b0720.id)
print('gene reaction: ', b0720.reactions)

gene name:  b0720
gene id:  b0720
gene reaction:  frozenset({<Reaction CS at 0x1b12c96f580>})


### Inspecting the systems' boundaries

* see the exchange fluxes
* see the objective function (the reaction set to be optimized)

Use print(model.summary())

You can also find the objective function using the following filtering technique:
* [r for r in model.reactions if r.objective_coefficient == 1]
* the reaction id of the biomass is Ec_biomass_iJO1366_WT_53p95M
and the exchange fluxes can be accessed using:
* model.boundary

In [35]:
## TODO
## Write your code below
# print(model.summary()) --> gives objective    Uptake      secretion
print(model.boundary[0]) # gives boundary reactions

objective_functions = [r for r in model.reactions if r.objective_coefficient == 1]
print(type(objective_functions))
objective_functions[0]

DM_4CRSOL: 4crsol_c --> 
<class 'list'>


Reaction identifier,Ec_biomass_iJO1366_WT_53p95M
Name,E. coli biomass objective function (iJO1366) - WT - with 53.95 GAM estimate
Memory address,0x1b12b6351b0
Stoichiometry,0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala_L_c + 0.000223 amet_c + 0.28742 arg_L_c +... 0.000223 10-Formyltetrahydrofolate + 0.000223 2-Demethylmenaquinol 8 + 2.5e-05 [2Fe-2S] iron-sulfur cluster + 0.000248 [4Fe-4S] iron-sulfur cluster + 0.000223 5-Methyltetrahydrofolate + 0.000279...
GPR,
Lower bound,0.0
Upper bound,1000.0


### Running a Flux Balance Analysis (FBA).

Documentation: [https://cobrapy.readthedocs.io/en/latest/simulating.html](https://cobrapy.readthedocs.io/en/latest/simulating.html)

By default, the model boundary condition (growth medium) is M9 aerobic (glucose minimal)

1.  Check the medium by inspecting the lower_bound of the following reactions:
  * EX\_glc\_e\_.lower_bound
  * EX\_o2\_e\_.lower_bound
2.  Optimize biomass using: 
  * solution = model.optimize()
  
3.  Inspect the solution as we did previously in Part 1.2 Optimization.



In [36]:
(model.reactions.EX_glc_e_)


Reaction identifier,EX_glc_e_
Name,D-Glucose exchange
Memory address,0x1b12b6beaa0
Stoichiometry,glc_D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


In [37]:
(model.reactions.EX_o2_e_)

Reaction identifier,EX_o2_e_
Name,O2 exchange
Memory address,0x1b12b6dace0
Stoichiometry,o2_e <=> O2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [38]:
solution = model.optimize()

print("Objective value: %.2f\n" % solution.objective_value)
print("Status: %s\n" % solution.status)

print("Fluxes:\n")
print(solution.fluxes)

# Converting the solution into a pandas dataframe
df = solution.to_frame()
# Saving the solution into tab-separed-value (tsv) format (plain text)
df.to_csv("out/iJO1366_fba.tsv", sep="\t")

Objective value: 0.96

Status: optimal

Fluxes:

DM_4CRSOL    0.000215
DM_5DRIB     0.000223
DM_AACALD    0.000000
DM_AMOB      0.000002
DM_MTHTHF    0.001293
               ...   
ZN2abcpp     0.000000
ZN2t3pp      0.000000
ZN2tpp       0.000313
ZNabcpp      0.000000
Zn2tex       0.000313
Name: fluxes, Length: 2583, dtype: float64


Inspect the flux value of the following reactions
* The glucose consumption: EX_glc_e_
* The oxygen consumption: EX_o2_e_
* The growth rate: Ec_biomass_iJO1366_WT_53p95M

HINT 1: use the solution object -> solution.fluxes.reaction_id <br>
HINT 2: use model.summary()

In [39]:
## TODO
## Write your code below
print('glucose consumption: ', solution.fluxes.EX_glc_e_)
print('oxygen consumption: ', solution.fluxes.EX_o2_e_)
# How to inspect the growth rate of Ec_biomass_iJO1366_WT_53p95M
print('growth rate: ', solution.objective_value)
print(model.summary())

glucose consumption:  -10.0
oxygen consumption:  -18.58848899696878
growth rate:  0.9646474389187335
Objective
1.0 Ec_biomass_iJO1366_WT_53p95M = 0.9646474389457608

Uptake
------
Metabolite      Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e_  0.004777         0  0.00%
    cbl1_e    EX_cbl1_e_ 0.0002151        62  0.02%
      cl_e      EX_cl_e_  0.004777         0  0.00%
 cobalt2_e EX_cobalt2_e_ 2.315E-05         0  0.00%
     cu2_e     EX_cu2_e_ 0.0006502         0  0.00%
     fe2_e     EX_fe2_e_   0.01498         0  0.00%
   glc_D_e     EX_glc_e_        10         6 99.98%
       k_e       EX_k_e_    0.1791         0  0.00%
     mg2_e     EX_mg2_e_  0.007961         0  0.00%
     mn2_e     EX_mn2_e_ 0.0006347         0  0.00%
    mobd_e    EX_mobd_e_ 0.0001322         0  0.00%
     nh4_e     EX_nh4_e_     10.13         0  0.00%
     ni2_e     EX_ni2_e_ 0.0002961         0  0.00%
      o2_e      EX_o2_e_     18.59         0  0.00%
      pi_e      EX_pi_e_    0.8944      

## Exercise 2.2: 

1. Change the oxygen exchange lower bound to zero to simulate anaerobic growth.
2. Optimize the model
3. What is the maximal growth rate in anaerobic conditions
4. what are the main three secretion products?

In [41]:
## TODO
## Write your code below
model.reactions.EX_o2_e_.lower_bound = 0
solution = model.optimize()
print("Objective value: %.2f\n" % solution.objective_value)
print("Status: %s\n" % solution.status)
# print(model.summary())



Objective value: 0.18

Status: optimal

Objective
1.0 Ec_biomass_iJO1366_WT_53p95M = 0.18399435353841975

Uptake
------
Metabolite      Reaction      Flux  C-Number C-Flux
     ca2_e     EX_ca2_e_ 0.0009111         0  0.00%
    cbl1_e    EX_cbl1_e_ 4.103E-05        62  0.00%
      cl_e      EX_cl_e_ 0.0009111         0  0.00%
     co2_e     EX_co2_e_    0.0633         1  0.11%
 cobalt2_e EX_cobalt2_e_ 4.416E-06         0  0.00%
     cu2_e     EX_cu2_e_  0.000124         0  0.00%
     fe2_e     EX_fe2_e_   0.00149         0  0.00%
     fe3_e     EX_fe3_e_  0.001367         0  0.00%
   glc_D_e     EX_glc_e_        10         6 99.89%
     h2o_e     EX_h2o_e_     3.805         0  0.00%
       k_e       EX_k_e_   0.03417         0  0.00%
     mg2_e     EX_mg2_e_  0.001519         0  0.00%
     mn2_e     EX_mn2_e_ 0.0001211         0  0.00%
    mobd_e    EX_mobd_e_ 2.521E-05         0  0.00%
     nh4_e     EX_nh4_e_     1.933         0  0.00%
     ni2_e     EX_ni2_e_ 5.649E-05         0  0.

In [61]:
(model.summary().uptake_flux)
model.summary()._objective_value
print('type(model.summary().secretion_flux --> ', type(model.summary().secretion_flux))
model.summary().secretion_flux

type(model.summary().secretion_flux -->  <class 'pandas.core.frame.DataFrame'>


,flux,reaction,metabolite
DM_4CRSOL,-4.103074e-05,DM_4CRSOL,4crsol_c
DM_5DRIB,-2.066257e-04,DM_5DRIB,5drib_c
DM_AACALD,0.000000e+00,DM_AACALD,aacald_c
DM_AMOB,-3.679887e-07,DM_AMOB,amob_c
DM_MTHTHF,-2.465524e-04,DM_MTHTHF,mththf_c
...,...,...,...
EX_xan_e_,0.000000e+00,EX_xan_e_,xan_e
EX_xmp_e_,0.000000e+00,EX_xmp_e_,xmp_e
EX_xtsn_e_,0.000000e+00,EX_xtsn_e_,xtsn_e
EX_xyl_D_e_,0.000000e+00,EX_xyl_D_e_,xyl_D_e


'optimal'

## Exercise 2.3: 

1. Set the oxygen exchange lower bound to -20
2. Set the glucose exchange flux (EX_glc_D_e_ lower bound to 0)
3. Set the glucose exchange flux (EX_ac_e_ lower bound to -10)

What is the maximal growth rate using acetate as soley carbon source
what is the oxygen uptake rate?

In [63]:
## TODO
## Write your code below
model.reactions.EX_o2_e_.lower_bound = -20
model.reactions.EX_glc_D_e_lower_bound = 0
model.reactions.EX_ac_e_lower_bound = 10

solution = model.optimize()
print('growth rate: ', solution.objective_value)


growth rate:  0.9646474389457961


## Optional Visualizing flux distributions using Escher

[Escher documentation](https://escher.readthedocs.io/en/latest/)

Escher online WebApp: [https://escher.github.io/](https://escher.github.io/#/)

In [ ]:
import escher
from escher import Builder

# Lets crate a builder by passing our model as well a given map name to tell escher how to represent the network
# Check the escher web to see other maps https://escher.github.io/#/
builder = Builder(organism='Escherichia coli', map_name='iJO1366.Central metabolism')
builder.reaction_data = solution.fluxes
builder

In [ ]:
# Add the optimal flux distribution to our map builder
model.reactions.EX_o2_e_.lower_bound = -20
model.reactions.EX_glc_e_.lower_bound = -10
model.reactions.EX_ac_e_.lower_bound = 0


gene =  model.genes.b0720
with model:
    gene.knock_out()
    ko_solution = model.optimize()

builder.reaction_data = ko_solution.fluxes
builder